In [1]:
#IMPORTING
import cv2
from skimage.filters import gaussian
from scipy.signal import firwin

from numpy.fft import fft, ifft
from numpy.fft import fft2, fftshift, ifft2, ifftshift
from numpy import tile, real, min, zeros
import numpy as np

import math
import openpyxl
import time

M_PI = math.pi
eps = 2**(-52)

START_whole = time.time()

In [2]:
############################## Parameter Initializing ################################
videoName = 'cantilever (2200fps)'
videoType = '.mp4'

Low_Freq = 350
High_Freq = 370
alpha = 5
Sampling_Rate = 2200

refFrame = 0
NumberOfPyramid = 2
Orientation = 2
sigma = 1
attenuateOtherFreq = 0 # 영역 외 주파수에 대해 베재할 것인가에 대해 True or False

In [3]:
################################# Mining Tools #####################################
def LOGGING(n):
    print('line {} passed'.format(n))

def timeTIC():
    START = time.time()
    return START

def timeTOK(n, START):
    print('process {} done / delay: {}'.format(n, time.time()-START))


def xlsxSave (savingName, data):
    #data = np.array(data)
    wb = openpyxl.Workbook()
    sheet = wb.active

    for x in range(data.shape[1]):
        for y in range(data.shape[0]):
            sheet.cell(row=x + 1, column=y + 1).value = data[y][x]

    wb.save('./data/{}.xlsx'.format(savingName))


In [4]:
########################## Filter Functions ######################

def getFilters(dimension, rVals, orientations):
    X = int(dimension[1])
    Y = int(dimension[0])

    defaultTwidth = 1

    twidth = defaultTwidth

    polarGrid = getPolarGrid(dimension)
    count = 0

    angle = np.array(polarGrid[0]) #픽셀 위치별 각도 값
    rad = np.array(polarGrid[1]) #픽셀 위치별 거리 값

    mask = getRadialMaskPair(rVals[0], rad, twidth)
    himask = mask[0]
    lomaskPrev = mask[1]



    filters = []
    filters.append(himask)

    for k in range(1, len(rVals)):
        mask = getRadialMaskPair(rVals[k], rad, twidth)
        himask = mask[0]
        lomask = mask[1]

        radMask = himask*lomaskPrev

        for j in range(1, orientations+1):
            angleMask = getAngleMask(j, orientations, angle)
            filters.append(radMask*angleMask/2)

        lomaskPrev = lomask
    filters.append(lomask)

    for k in range(len(filters)):
        filters[k] = np.array(filters[k])
    return filters

def getPolarGrid(dimension):
    X = dimension[1]
    Y = dimension[0]
    centerX = int(X / 2)
    centerY = int(Y / 2)

    # Create rectangular grid
    xramp = np.array([ [(x-int(X/2))/(X/2) for x in range(X)] for y in range(Y)])
    yramp = np.array([ [(y-int(Y/2))/(Y/2) for x in range(X)] for y in range(Y)])
    angle = np.arctan2(xramp, yramp)+M_PI/2

    rad = np.sqrt(xramp**2+yramp**2)
    rad[centerY][centerX] = rad[centerY-1][centerX]

    polarGrid = [angle, rad]
    return polarGrid


def getRadialMaskPair(r, rad, twidth): #3차원 그래프상으로 보이는 완전 꼬깔 형태에서, 위 아래의 범위를 잘라내고 원뿔대 같은 형태도 만든다.
    X = int(rad.shape[1])
    Y = int(rad.shape[0])

    log_rad = np.log2(rad)-np.log2(r)

    himask = log_rad
    himask[himask>0] = 0
    himask[himask<-twidth] = -twidth
    himask = himask*M_PI/(2*twidth)

    himask = np.cos(himask)
    lomask = np.sqrt(1-himask**2)

    mask = [himask, lomask]
    return mask

def getAngleMask(b, orientations, angle):
    order = orientations - 1
    const = (2 ** (2 * order)) * (math.factorial(order) ** 2) / (orientations * math.factorial(2 * order)) # Scaling constant

    angle_ = (M_PI + angle - (M_PI * (b - 1) / orientations)) % (2 * M_PI) - M_PI
    anglemask = 2 * np.sqrt(const) * (np.cos(angle_) ** order) * (abs(angle_) < (M_PI / 2))  # Make falloff smooth
    return anglemask

def getFilterIDX2(filters, orientations, rVals):
    X = filters[0].shape[1]
    Y = filters[0].shape[0]
    nFilts = len(filters)
    filtIDX = [[None for j in range(orientations)] for i in range(nFilts)]
    croppedFilters = []

    #himask IDX
    filtIDX[0][0] = [y for y in range(Y)]
    filtIDX[0][1] = [x for x in range(X)]
    croppedFilters.append(filters[0])

    #stearable filter IDX
    for k in range(1, nFilts-1, orientations):
        n = int(k/2)+1
        lb_y = int( (Y*(sum(rVals[0:n])-1))/2 )
        ub_y = Y - lb_y
        lb_x = int( (X*(sum(rVals[0:n])-1))/2 )
        ub_x = X - lb_x

        for i in range(orientations):
            filtIDX[k+i][0] = [y + lb_y for y in range(ub_y - lb_y)]
            filtIDX[k+i][1] = [x + lb_x for x in range(ub_x - lb_x)]

        for i in range(orientations):
            croppedFilters.append(filters[k+i][lb_y:ub_y, lb_x:ub_x])


    #lomaskIDX
    lb_y = int( (Y * (sum(rVals) - 1))/2 )
    ub_y = Y - lb_y
    lb_x = int( (X * (sum(rVals) - 1))/2 )
    ub_x = X - lb_x


    filtIDX[nFilts - 1][0] = [y + lb_y for y in range(ub_y - lb_y)]
    filtIDX[nFilts - 1][1] = [x + lb_x for x in range(ub_x - lb_x)]
    croppedFilters.append(filters[nFilts-1][lb_y:ub_y, lb_x:ub_x])

    filterIDX = [croppedFilters, filtIDX]
    return filterIDX


In [5]:

def fir_window_bp_2(delta, fl, fh):
    length = delta.shape[0]+1
    b = firwin(length, (fl * 2, fh * 2), pass_zero=False)[0:length-1]

    m = delta.shape[1]
    batches = 20
    batch_size = int(m / batches) + 1
    temp = fft(ifftshift(b))
    out = zeros(delta.shape, dtype=delta.dtype)

    i=0
    indexes = (batch_size * i, min((batch_size * (i + 1), m)))
    while (indexes[0]<m):
        indexes = (batch_size * i, min((batch_size * (i + 1), m)))
        freq = fft(delta[:, indexes[0]:indexes[1]], axis=0) * tile(temp, (
            delta.shape[2], indexes[1] - indexes[0], 1)).swapaxes(0, 2)
        out[:, indexes[0]:indexes[1]] = real(ifft(freq, axis=0))
        i += 1
        indexes = (batch_size * i, min((batch_size * (i + 1), m)))
    return out

def amplitude_weighted_blur(x, weight, sigma):
    if sigma != 0:
        return gaussian(x*weight, sigma, mode="wrap") / gaussian(weight, sigma, mode="wrap")
    return x

In [6]:
################### Color Channel Functions #####################
# OpenCV에서 따로 YIQ관련 변환 알고리즘을 지원하지 않아서 mathwork 공식 계산 수치들을 들고와 대입했다.
# HSV가 V가 grayscale과 같은 방식으로 명암을 저장하기 때문에, YIQ의 Y와 동일한 역할을 한다. 
# 그러나 Hue, Saturation에 대한 부분 때문인지 HSV의 경우는 원본 영상의 형상이 뚜렷하게 남아있어 결과물이 이상하다.

# Y는 Luminescence로 Brightness(명암)을 의미한다.
# I는 red~cyan의 hue(색조)값을 의미한다, Q는 green~magenta의 hue 값을 의미한다.
# 즉 HSV와 달리 saturation(채도)는 취급하지 않으며, hue를 사이클 적으로 다르는 HSV와 달리 두개의 boundary가 뚜렷한 색조로써 취급한다.
# 이에 따라 배포 코드에서 제시한 YIQ 좌표를 따른다.

def bgr2yiq(img):
    R = img[:, :, 2]
    G = img[:, :, 1]
    B = img[:, :, 0]

    Y = 0.299 * R + 0.587 * G + 0.114 * B
    I = 0.596 * R - 0.274 * G - 0.322 * B
    Q = 0.211 * R - 0.523 * G + 0.312 * B

    img_yiq = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_yiq[:, :, 0] = Y
    img_yiq[:, :, 1] = I
    img_yiq[:, :, 2] = Q
    #YIQ = np.array([[0.299, 0.587, 0.114],[0.596, -0.274, -0.322],[0.211, -0.523, 0.312]])
    return img_yiq

def yiq2bgr(img):
    Y = img[:, :, 0]
    I = img[:, :, 1]
    Q = img[:, :, 2]

    R = Y+0.956*I+0.621*Q
    G = Y-0.272*I-0.647*Q
    B = Y-1.106*I+1.703*Q

    img_rgb = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    img_rgb[:, :, 0] = B
    img_rgb[:, :, 1] = G
    img_rgb[:, :, 2] = R
    #YIQ = np.array([[0.299, 0.587, 0.114],[0.596, -0.274, -0.322],[0.211, -0.523, 0.312]])
    return img_rgb

In [7]:
START = timeTIC()

filename = './input/{}{}'.format(videoName, videoType)

cap_original = cv2.VideoCapture(filename)

####################################### RGB to Gray Scale (save)
ret, frame = cap_original.read()
frameX = frame.shape[1]
frameY = frame.shape[0]
nColor = frame.shape[2]
nFrame = 0

while(True):
    if ret == False:
        break
    else:
        nFrame += 1
    ret, frame = cap_original.read()

cap_original.release()

pyrLayers = 2
rVals = [1]
for i in range(pyrLayers):
    rVals.append(0.5**(i+1))


####################################### Create Steerable Filter##########
filters = getFilters([frameY, frameX], rVals, Orientation)
filterIDX = getFilterIDX2(filters, Orientation, rVals)
croppedFilters = filterIDX[0]
filtIDX = filterIDX[1]


###########################Initialization of motion magnified luma component

magnifiedLumaFFT = np.zeros([nFrame, frameY, frameX], dtype=np.complex64)



timeTOK('0.Create Stearable Filter, Initializing', START)

process 0.Create Stearable Filter, Initializing done / delay: 0.4740269184112549


In [8]:
START = timeTIC()

################################ Spatial Fourier Transform ########################################
numLevels = len(filters)
vidFFT = np.zeros([nFrame,frameY,frameX], dtype=np.complex64)
# vidFFT[0] = saturation, vidFFT[1] = value

cap_original = cv2.VideoCapture(filename)
ret, frame = cap_original.read()

for k in range(0, nFrame):
    clipMat = frame/255
    tVid = bgr2yiq(clipMat)[:,:,0]
    vidFFT[k] = fftshift(fft2(tVid))
    ret, frame = cap_original.read()


cap_original.release()


timeTOK('1.Spatial Fourier Transform', START)


process 1.Spatial Fourier Transform done / delay: 8.347477436065674


In [9]:
START = timeTIC()

##################################Compute Phase Of level#######
for level in range(1, numLevels-1):
    X = len(croppedFilters[level][0])
    Y = len(croppedFilters[level])
    lb_x = filtIDX[level][1][0]
    ub_x = filtIDX[level][1][-1]+1
    lb_y = filtIDX[level][0][0]
    ub_y = filtIDX[level][0][-1]+1


    clipMat = croppedFilters[level] * vidFFT[refFrame][lb_y:ub_y, lb_x:ub_x]  # MATLAB과는 좌우 반전
    pyrRef = ifft2(ifftshift(clipMat))
    pyrRefPhaseOrig = pyrRef / abs(pyrRef)
    pyrRef = np.angle(pyrRef)

    delta = np.zeros([nFrame,Y,X], dtype=np.float16)
    matCheck = []
    for frameIDX in range(0, nFrame):
        filterResponse = ifft2(ifftshift( croppedFilters[level] * vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] ))
        pyrCurrent = np.angle(filterResponse)
        clipMat1 = pyrCurrent - pyrRef
        clipMat2 = M_PI + clipMat1
        clipMat3 = clipMat2%(2*M_PI)
        clipMat4 = clipMat3 - M_PI
        clipMat = clipMat4
        delta[frameIDX] = clipMat


    ####### Temporal Filtering
    delta_1 = fir_window_bp_2(delta, Low_Freq / Sampling_Rate, High_Freq / Sampling_Rate)  # Finite Impulse Response filter


    ####### Apply magnification
    for frameIDX in range(0, nFrame):
        Phase = delta_1[frameIDX]

        originalLevel = ifft2(ifftshift(croppedFilters[level]*vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]))

        if (sigma != 0):
            Phase = amplitude_weighted_blur(Phase, abs(originalLevel)+eps, sigma)

        Phase = alpha*Phase

        if (attenuateOtherFreq):
            tempOrig = abs(originalLevel)*pyrRefPhaseOrig
        else:
            tempOrig = originalLevel

        tempTransformOut = np.exp(1j*Phase)*tempOrig


        A = croppedFilters[level]
        B = fftshift(fft2(tempTransformOut))
        curLevelFrame = 2 * A * B

        matClip = magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]
        magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] = matClip + curLevelFrame


    print('level {} succeeded'.format(level))

    
timeTOK('2~4.Compute Phase of Level~', START)

level 1 succeeded
level 2 succeeded
level 3 succeeded
level 4 succeeded
process 2~4.Compute Phase of Level~ done / delay: 70.05500674247742


In [10]:
START = timeTIC()


##################################Add unmolested lowpass residual#####
level = len(filters)-1
lb_x = filtIDX[level][1][0]
ub_x = filtIDX[level][1][-1] + 1
lb_y = filtIDX[level][0][0]
ub_y = filtIDX[level][0][-1] + 1
for frameIDX in range(0, nFrame):
    lowpassFrame = vidFFT[frameIDX][lb_y:ub_y, lb_x:ub_x]*(croppedFilters[level]**2)
    matClip = magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] + lowpassFrame
    magnifiedLumaFFT[frameIDX][lb_y:ub_y, lb_x:ub_x] = matClip

##########################Saving
cap_original = cv2.VideoCapture(fileName)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
obj_out = cv2.VideoWriter('./output/(color) '+ str(Low_Freq) + '~' + str(High_Freq) + ', a=' + str(alpha) + ' {}.mp4'.format(videoName)
                          , fourcc, 30.0, (frameX, frameY))

ret, frame = cap_original.read()

for k in range(0, nFrame):
    clipMat1 = magnifiedLumaFFT[k]
    clipMat = ifft2(ifftshift(clipMat1))
    magnifiedLuma = np.real(clipMat)
    #명암에 대한 데이터만 적용하기 위해 yiq변환해서 Y 값만 가져온다.
    clipMat2 = bgr2yiq(frame)
    clipMat3 = magnifiedLuma*255
    clipMat2[:,:,0] = clipMat3
    
    #그 외의 Q, I는 그대로 사용할 것이므로 변환된 Y만 원본 영상에 대입한 뒤 다시 RGB로 변환한다.
    outFrame = yiq2bgr(clipMat2)
    
    outFrame[outFrame > 255] = 255
    outFrame[outFrame < 0] = 0
    outFrame = np.uint8(outFrame)
    #outFrame = np.uint8(yiq2bgr(clipMat2))

    obj_out.write(outFrame)
    ret, frame = cap_original.read()


timeTOK('5. Final Process', START)

process 5. Final Process done / delay: 12.062689781188965


In [11]:
cap_original.release()
obj_out.release()

print('Whole Process Finished / delay: {}'.format(time.time()-START_whole))

Whole Process Finished / delay: 91.13321232795715
